In [14]:
# Dependencies and Setup
import pandas as pd

# File to Load
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

purchase_data.tail(5)

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
775,775,Aethedru70,21,Female,60,Wolf,3.54
776,776,Iral74,21,Male,164,Exiled Doomblade,1.63
777,777,Yathecal72,20,Male,67,"Celeste, Incarnation of the Corrupted",3.46
778,778,Sisur91,7,Male,92,Final Critic,4.19
779,779,Ennrian78,24,Male,50,Dawn,4.60


### Player Count

Display the total number of players


In [15]:
# Pass the unique elements in "SN" series to len to get number of elements
uniquePlayers = len(pd.unique(purchase_data["SN"]))

print(f'{uniquePlayers} players.')


576 players.


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, total purchases, and total revenue.

* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [16]:
# Construct dataframe with the above quantifiers
summary_df = pd.DataFrame(
    {
        'Unique Items': len(pd.unique(purchase_data['Item ID'])),
        'Average Price': purchase_data['Price'].mean(),
        'Total Purchases': len(purchase_data['Price']),
        'Total Revenue': purchase_data['Price'].sum()
    },
    index=["Heroes of Pymoli Item Marketplace"]
)

# format the column data to be more readable
format_mapping = {"Average Price": "${:,.2f}","Total Revenue": "${:,.2f}"}

summary_df.style.format(format_mapping)

,Unique Items,Average Price,Total Purchases,Total Revenue
Heroes of Pymoli Item Marketplace,179,$3.05,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [11]:
# Construct 3 new dataframes based on the possible gender options
male_df = purchase_data[purchase_data["Gender"]=="Male"]

female_df = purchase_data[purchase_data["Gender"]=="Female"]

other_df = purchase_data[(purchase_data["Gender"]!="Female")&(purchase_data["Gender"]!="Male")]

In [17]:
# Define a function that calculates a percent of purchases for the item passed
def percent(count):
    percent = round(100 * (count / int(len(purchase_data['Price']))),2)
    return percent

In [18]:
# DataFrame with percentages and count of each gender option
genderDemo = pd.DataFrame(
    {
        "Percentage": [
            percent(len(male_df)),
            percent(len(female_df)),
            percent(len(other_df))
        ],
        "Count": [
            len(male_df),
            len(female_df),
            len(other_df)
        ]
    }, index = ["Male","Female","Other"]
)

genderDemo["Percentage"] = genderDemo["Percentage"].map("{:,.2f}%".format)
genderDemo


,Percentage,Count
Male,83.59%,652
Female,14.49%,113
Other,1.92%,15



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [19]:
# Dataframe with price data from each gender dataframe
genderSummary = pd.DataFrame(
    {
        "Purchase Count": [ #how many purchases per gender option
            len(female_df),
            len(male_df),
            len(other_df)
                          ],
        "Average Purchase": [ 
            female_df["Price"].mean(),
            male_df["Price"].mean(),
            other_df["Price"].mean()
                            ],
        "Total Purchase Value": [
            female_df["Price"].sum(),
            male_df["Price"].sum(),
            other_df["Price"].sum()
                                ],
        "Average Purchase Total per Person": [ 
            #for each SN, find out how much they spent, 
            #then find the average over all SN's in the gender df
            female_df.groupby(["SN"])["Price"].sum().mean(),
            male_df.groupby(["SN"])["Price"].sum().mean(),
            other_df.groupby(["SN"])["Price"].sum().mean()
        ]
        
    }, index=['female','male','other'] 
)                               

In [20]:
# Formatting to prettify 
genderSummary['Average Purchase'] = genderSummary['Average Purchase'].map("${:,.2f}".format)
genderSummary['Average Purchase Total per Person'] = genderSummary['Average Purchase Total per Person'].map("${:,.2f}".format)
genderSummary['Total Purchase Value'] = genderSummary['Total Purchase Value'].map("${:,.2f}".format)
genderSummary
              

,Purchase Count,Average Purchase,Total Purchase Value,Average Purchase Total per Person
female,113,$3.20,$361.94,$4.47
male,652,$3.02,"$1,967.64",$4.07
other,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [21]:
# Creates the bin based on the dataframe series and chosen bin size
def binConstructor(dataframe, series, bins=4):
    minSeries = dataframe[series].min() -1 #-1 to ensure the lower bound includes the smallest possible value
    maxSeries = dataframe[series].max() +1 #+1 to ensure this data set bins are whole numbers
    dataRange = maxSeries - minSeries
    output = [minSeries]
    for i in range(1,bins+1):
        binRight = i*(dataRange/bins) + minSeries
        output.append(binRight)
        
    return output

In [22]:
# Appends a bin column to the purchase_data df
binAge = binConstructor(purchase_data,"Age",8)

age_df = purchase_data
age_df["Bin"] = pd.cut(age_df["Age"],binAge) 
binned_df = age_df.groupby(age_df["Bin"])

In [48]:
# Count the number of purchases in each age bin
player_count = binned_df["SN"].count()

# dataframe to display count and percent of purchases in each bin
demo_final = pd.DataFrame(
    {
        "Player Count": player_count,
        "Percentage": 100 * player_count / player_count.sum(),
    }
)

In [49]:
# Format to percent
demo_final["Percentage"] = demo_final["Percentage"].map("{:,.2f}%".format)
demo_final

,Player Count,Percentage
Bin,,
"(6.0, 11.0]",39,5.00%
"(11.0, 16.0]",77,9.87%
"(16.0, 21.0]",232,29.74%
"(21.0, 26.0]",277,35.51%
"(26.0, 31.0]",70,8.97%
"(31.0, 36.0]",50,6.41%
"(36.0, 41.0]",30,3.85%
"(41.0, 46.0]",5,0.64%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [38]:
# creates a new dataframe with data grouped by age demographic
purchases_df = pd.DataFrame(
    {
        "Purchase Count": age_df["Bin"].value_counts(),
        "Average Purchase Price":binned_df["Price"].mean(),
        "Total Purchase Value":binned_df["Price"].sum(),
        "Average Purchase Total":binned_df["Price"].sum() / age_df["Bin"].value_counts()
    },
    index = pd.unique(age_df["Bin"])
)

In [39]:
# Dataframe formatting
format_mapping = {
    "Average Purchase Price": "${:,.2f}",
    "Total Purchase Value": "${:,.2f}",
    "Average Purchase Total":"${:,.2f}"
}
purchases_df.sort_values("Purchase Count",ascending=False).style.format(format_mapping)

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Purchase Total
"(21.0, 26.0]",277,$3.04,$841.09,$3.04
"(16.0, 21.0]",232,$3.07,$711.74,$3.07
"(11.0, 16.0]",77,$2.97,$228.37,$2.97
"(26.0, 31.0]",70,$2.93,$205.28,$2.93
"(31.0, 36.0]",50,$2.89,$144.32,$2.89
"(6.0, 11.0]",39,$3.28,$127.75,$3.28
"(36.0, 41.0]",30,$3.54,$106.23,$3.54
"(41.0, 46.0]",5,$3.00,$14.99,$3.00


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [40]:
# Remove the bin column as it is not needed anymore
purchase_data = purchase_data.drop(columns="Bin")

In [41]:
# create a dataframe which shows purchase count, avg price, and total price by SN
topSpender_df = pd.DataFrame(
    {
        "Purchase Count": purchase_data["SN"].value_counts(),
        "Average Purchase Price": purchase_data.groupby("SN")["Price"].mean(),
        "Total Purchase Value": purchase_data.groupby("SN")["Price"].sum()
    }
)

In [42]:
# Formatting and sorting dataframe
format_mapping = {
    "Average Purchase Price": "${:,.2f}",
    "Total Purchase Value": "${:,.2f}"
}
topSpender_df.sort_values(by="Total Purchase Value",ascending=False).head(5).style.format(format_mapping)

,Purchase Count,Average Purchase Price,Total Purchase Value
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, average item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [43]:
# Reduce the size of df to what is needed
item_id = purchase_data[["Item ID","Item Name","Price"]]
# group by id and name
item_group = item_id.groupby(["Item ID","Item Name"])

itemCount = item_id.groupby("Item ID")["Item Name"].value_counts()
itemPrice = item_group["Price"].mean()
itemTotal = itemCount * itemPrice

In [44]:
# Construct dataframe from above calculations
popular_df = pd.DataFrame(
    {
        "Purchase Count": itemCount,
        "Item Price": itemPrice,
        "Total Purchase Value": itemTotal 
    }
)

In [45]:
# Formatting and sorting of df
format_mapping = {
    "Item Price": "${:,.2f}",
    "Total Purchase Value": "${:,.2f}"
}

popular_df.sort_values("Purchase Count",ascending=False).head(5).style.format(format_mapping)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [46]:
popular_df.sort_values("Total Purchase Value",ascending=False).style.format(format_mapping)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
59,"Lightning, Etcher of the King",8,$4.23,$33.84
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
78,"Glimmer, Ender of the Moon",7,$4.40,$30.80
72,Winter's Bite,8,$3.77,$30.16
